# Логгирование

Логгирование - очень полезный зверь. Все отладочные "принты" можно обернуть в логи, и вот у вас уже не куча неудобных и компроментирующих строк текста, а управляемая информация, которую при необходимости можно отключить или записать в файл. 

## Конфигурация логгера

В python есть модуль [`logging`](https://docs.python.org/3/howto/logging.html). Это инструмент для красивой записи логов как в файл, так и в терминал. 

Конфигурация определяет, какая информация будет отражена в логе. 

Уровни логов располагаются в следующей последовательности (чем ниже уровень, тем круче его нрав)

* DEBUG 
* INFO 
* WARNING 
* CRITICAL
* ERROR 

In [ ]:
import logging 

logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

Например, здесь

```python
level=logging.INFO
```

в конфиге указано, что будут отображаться все сообщения, начиная с уровня `INFO` и ниже, а именно:

* `WARNING`
* `ERROR`
* `CRITICAL`

При этом уровень `DEBUG` отображаться не будет.

Дальше идёт формат:

```python
format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
```

Он описывает форматирование лога. В примере указано, что сначала в строке пишутся дата и время, затем имя логгера, уровень лога и само сообщение. 

Формат логгирования можно создавать под себя, не обязательно он должен быть только таким.

Для каждого из уровней есть соответствующий метод

In [ ]:
logging.debug('This is a debug message')
logging.info('This is an info message')
logging.warning('This is a warning message')
logging.error('This is an error message')
logging.critical('This is a critical message')

Дату и время в начале сообщений мы видим как раз из-за формата, который мы прописали в конфиге.

## Класс с логгером

Давайте создадим класс, внутри которого будет его личный логгер. 

In [ ]:
class Point:
    point_type = "2D"
    
    def __init__(self, x: float, y: float) -> None:
        self._logger = logging.getLogger("point_logger")
        
        self.x = x 
        self.y = y 

        self._logger.info("Point was created successfully!")
        self._logger.debug("Debug information!")
    
    def get_coordinates(self) -> tuple:
        return (self.x, self.y)

При создании логгера, его имя можно написать статической строкой, например "point_logger". 

In [ ]:
point = Point(0, 0)

Так как у нас выставлен уровень логгирования: `INFO`, то уровень `DEBUG` показан не будет. 

Так бывает, что в ходе рефакторинга классы переименовываются, и в нашем случае имя логгера тоже придётся менять руками. Что лень и не всегда можно заметить. Поэтому есть более автоматический способ, как именовать логгер. 

In [ ]:
class Point:
    point_type = "2D"
    
    def __init__(self, x: float, y: float):
        self._logger = logging.getLogger(self.__class__.__name__)
        
        self.x = x 
        self.y = y 

        self._logger.info("Point was created successfully!")
        self._logger.debug("Debug information!")
    
    def get_coordinates(self) -> tuple:
        return (self.x, self.y)

In [ ]:
point = Point(0, 0)

В этом случае логгер будет всегда иметь имя класса. Если вы поменяете имя класса, то и имя логгера изменится. Das ist удобно! 

По умолчанию логгер будет писать в стандартный вывод/терминал, но можно перенаправить его, например, в файл. Всё это делается через конфиг: 

In [ ]:
logging.basicConfig(filename="logs")

## "Как бы" приватность

В питоне есть договорённость: если нужно показать, что атрибут приватный, то его имя должно начинаться с нижнего подчёркивания. Но при этом, к нему всё ещё можно будет обратиться снаружи, так что это скорее признак хорошего тона. 

Логгер - это штука внутренняя, которая является личным делом каждого класса, поэтому хорошей практикой является делать его "как бы" приватным.

```python
self._logger = getLogger(self.__class__.__name__)
```

In [ ]:
point._logger.info("Hello!")

Если хочется чуть больше приватности, то можно добавить два нижних подчёркивания. Тогда уже так просто не получится обратиться к атрибуту извне (язык Питон диктует свои правила). 

Но в этом случае делать так не рекомендую, так как в Python по-настоящему ничего нельзя спрятать (ограничить доступ). И два нижних подчёркивания будут только сбивать с толку людей, читающих ваш код. 

In [ ]:
class Point:
    def __init__(self, x: float, y: float) -> None:
        self.__logger = logging.getLogger(self.__class__.__name__)
        self.x = x 
        self.y = y 

        self.__logger.info("Point was created successfully!")
    
    def get_coordinates(self) -> tuple:
        area = self.__calculate_area(self)
        return (self.x, self.y, area)

Итак, мы сделали "как бы" приватный логгер. Давайте попробуем к нему обратиться "как обычно" - через точку. 

In [ ]:
point = Point(0, 0)

point.__logger.info("Hello!")

Та-дам! Упала ошибка, которая говорит нам, что в классе Point нет такого логера. В целом, вот оно какое "ограничение доступа" в питоне. 

НО! Главное помнить, что в python НЕТ ПРИВАТНОСТИ! Если очень захотеть, то всё равно можно обратиться ^____^ 

In [ ]:
point._Point__logger.info("Hello")

Такая запись продиктована метаклассами в языке Python. Почему так и зачем, лучше спросить у создателей языка. 

И как вы можете видеть, мы легко и играючи обратились к "как бы приватному" аттрибуту класса. Так что, привыкайте к питонячному эксгибиционизму - все и всегда всё видят (=

## PEP

В Python для всего есть набор правил (рекомендаций) - PEP = Python Enhancement Proposal. 

Для логгирования, конечно же, тоже есть - [PEP282](https://www.python.org/dev/peps/pep-0282/).

Не бойтесь просматривать такие файлы, это не сухие ГОСТы, а документация, написанная человеческим языком. 